In [1]:
# ruff: noqa: E402 F401
# https://www.kaggle.com/code/akshitsharma1/cat-vs-dog-easy-resnet-tutorial-for-beginners
%matplotlib inline

# Importing necessary libraries and modules
import copy
import glob
import os  # for interacting with the operating system and working with files
import random
import shutil
import time
import zipfile
from pathlib import Path

import matplotlib.pyplot as plt  # for creating visualizations
import numpy as np  # for numerical computing
import pandas as pd  # for manipulation and analysis
import torch  # for machine learning
import torch.nn as nn  # for building neural networks in PyTorch
import torch.optim as optim  # for optimizing neural networks in PyTorch
import torchvision  # for computer vision tasks in PyTorch
import torchvision.models as models  # for using pre-trained models in PyTorch
from PIL import Image  # for working with images
from torch.utils.data import DataLoader, Dataset  # for working with datasets in PyTorch
from torchvision import models, transforms  # for loading and preprocessing image data

# Set the device (cuda if available, otherwise cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define paths
url = "https://s3.amazonaws.com/fast-ai-sample/dogscats.tgz"
download_path = Path.home() / ".fastai/archive/dogscats.tgz"
path = Path.home() / ".fastai/data/dogscats"  # Change to the actual path where you download and extract the dataset
model_checkpoint_path = Path("model_checkpoints")
model_checkpoint_path.mkdir(parents=True, exist_ok=True)

# Download and unpack dataset
!wget -c {url} -O {str(download_path)}
if not path.parent.is_dir():
    shutil.unpack_archive(download_path, path.parent)

--2024-01-03 08:36:50--  https://s3.amazonaws.com/fast-ai-sample/dogscats.tgz
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.105.141, 52.217.16.78, 52.217.75.94, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.105.141|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [23]:
# Getting the file paths for the training and testing images
train_path: Path = path / "train"
train_x = list((train_path / "dogs").iterdir()) + list((train_path / "cats").iterdir())
valid_path: Path = path / "valid"
valid_x = list((train_path / "dogs").iterdir()) + list((train_path / "cats").iterdir())
test_x = list((path / "test1").iterdir())
assert len(train_x) > 0
assert len(valid_x) > 0
assert len(test_x) > 0


# Defining a custom dataset class for loading and preprocessing image data
class ManualDataset(Dataset):

    def __init__(self, files, split="train", val_split=False):
        self.raw_files = files
        self.split = split
        self.val_split = val_split

        # Defining the transformations to be applied to the images
        self.train_transform = transforms.Compose(
            [transforms.Resize((224, 224)),
             transforms.RandomHorizontalFlip(),
             transforms.ToTensor()]
        )
        self.test_transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

        # Splitting the data into training and testing sets
        if split == "test":
            if val_split:
                self.raw_files = self.raw_files[:10]
            else:
                self.raw_files = self.raw_files[10:]

    def __len__(self):
        return len(self.raw_files)

    def __getitem__(self, idx):
        # Reading in the image file
        raw_file: Path = self.raw_files[idx]
        raw = Image.open(raw_file)

        # Applying the appropriate transformation based on the split
        if self.split == "train":
            raw = self.train_transform(raw)
        elif self.split == "test":
            raw = self.test_transform(raw)

        # Extracting the label from the file name
        label = raw_file.name.split(".")[0]
        # label = raw_file.split("/")[-1].split(".")[0]
        if label == "dog":
            label = 1
        elif label == "cat":
            label = 0

        return raw, label

In [24]:
# Creating instances of the ManualDataset class for the training, testing, and validation datasets
train_dataset = ManualDataset(train_x, split="train")
test_dataset = ManualDataset(test_x, split="test")
val_dataset = ManualDataset(train_x, split="test", val_split=True)

# Defining the batch sizes for the training and testing data
train_batch_size = 32
test_batch_size = 1

# Creating instances of the DataLoader class for the training, testing, and validation data
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size)
val_loader = DataLoader(val_dataset, batch_size=test_batch_size)

In [25]:
resnet50 = models.resnet50(pretrained=True)

# Remove the last fully connected layer (fc) to customize the model
num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential()

# Add custom layers for binary classification
custom_classifier = nn.Sequential(nn.Linear(num_features, 512), nn.ReLU(), nn.Linear(512, 1), nn.Sigmoid())

# Attach the custom classifier to the ResNet50 model
resnet50.fc = custom_classifier

# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet50.to(device)

/home/burny/.cache/pypoetry/virtualenvs/deep-learning-examples-d9n-7CCf-py3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/burny/.cache/pypoetry/virtualenvs/deep-learning-examples-d9n-7CCf-py3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [26]:
param_to_optimize = []
for name, param in resnet50.named_parameters():
    print(name)
    if ('fc' in name):
        param.requires_grad = True
        param_to_optimize.append(param)
    else:
        param.requires_grad = False

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.conv3.weight
layer1.2.bn3.weight
layer1.2.bn3.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.conv3.weight
layer2.0.bn3.weight
layer2.0.bn3.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.we

In [27]:
# Defining the loss function and optimizer for training the model
criterion = nn.BCELoss()
optimizer = optim.SGD(params=param_to_optimize, lr=0.001, momentum=0.9)

# Defining the number of epochs to train the model
epochs = 10

# Setting the model to training mode
resnet50.train()

# Training the model for the specified number of epochs
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    # Iterating through the training data
    for data, label in train_loader:
        data = data.to(device)
        label = label.to(device)
        label = label.type(torch.float)
        output = resnet50(data)
        label = label.unsqueeze(1)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += (loss.item()) / len(train_loader)

    print('Epoch : {},  train loss : {}'.format(epoch + 1, epoch_loss))

    # Setting the model to evaluation mode
    resnet50.eval()
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0

        # Iterating through the validation data
        for data, label in val_loader:
            data = data.to(device)
            label = label.to(device)
            label = label.type(torch.float)
            output = resnet50(data)
            label = label.unsqueeze(1)

            val_output = resnet50(data)
            val_loss = criterion(val_output, label)

            epoch_val_loss += val_loss / len(val_loader)

        print('Epoch : {} , val_loss : {}'.format(epoch + 1, epoch_val_loss))

KeyboardInterrupt: 

In [ ]:
# Creating empty lists to store the IDs and predictions
id_list = []
pred_list = []

# Setting the model to evaluation mode
resnet50.eval()

# Disabling gradient calculation to save memory
with torch.no_grad():
    # Iterating through the test data
    for test_path in (test_x):
        # Opening the image and extracting the ID
        img = Image.open(test_path)
        _id = int(test_path.split('/')[-1].split('.')[0])

        # Applying the test transformations to the image
        test_transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
        img = test_transform(img)
        img = img.unsqueeze(0)
        img = img.to(device)

        # Computing the output of the model and converting it to a list
        outputs = (resnet50(img)).tolist()

        # Adding the ID and prediction to the respective lists
        id_list.append(_id)
        pred_list.append(outputs[0])

# Converting the prediction list to a numpy array and rounding it to the nearest integer
pred_list = np.array(pred_list)
pred_list = np.round(pred_list.flatten()).astype(int)

# Creating a DataFrame with the IDs and predictions
res = pd.DataFrame({'id': id_list, 'label': pred_list})

# Sorting the DataFrame by ID and resetting the index
res.sort_values(by='id', inplace=True)
res.reset_index(drop=True, inplace=True)

# Saving the DataFrame to a CSV file
res.to_csv('submission.csv', index=False)